Esta es una implementación de una máquina de aprendizaje extremo.
Sobre los datos, basados en el dataset "Facial expresión recognition challenge" disponible en https://www.kaggle.com/datasets/debanga/facial-expression-recognition-challenge a través de una VQ-VAE se obtienen las matrices $x$ que contienen los índices de los embeddings a los que la imagen fue asociada en el espacio latente. La idea esa usar esas matrices para determinar si existe una correlación entre los índices de los embeddings seleccionados y la emoción a la que pertenece la clase.

In [ ]:
# Librerías
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, ClassifierMixin
import pickle
import warnings
warnings.filterwarnings('ignore')

Notas: 
1. Los datos estaban muy desbalanceados.
2. Los datos de entrenamiento y prueba que aquí se usan se obtuvieron de balancear el conjunto y separar en 80 - 20. Se hizo de esta forma para poder hacer una comparación con el mismo conjunto de datos en todos los modelos y que así no hubiera ninguna influencia sobre los datos de entrenamiento y prueba que le pudiera tocar a cada modelo.

In [2]:
x_train = np.load("x_train.npy")
x_test = np.load("x_test.npy")
y_train = np.load("y_train.npy")
y_test = np.load("y_test.npy")

# Para el perceptrón.
Se ajustó un MLP para usarlo como baseline en las comparativas de los resultados contra el ELM.

In [3]:
# Modelo para el perceptrón

model_perceptron = Sequential([
    Input(shape=(144,)),
    Dense(128, activation='sigmoid'),
    Dense(64, activation='sigmoid'),
    Dense(32, activation='sigmoid'),
    Dense(7, activation='softmax')
])

model_perceptron.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model_perceptron.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │        18,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 7)              │           231 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,127 (113.78 KB)

 Trainable params: 29,127 (113.78 KB)

 Non-trainable params: 0 (0.00 B)

In [4]:
# checkpoint para el perceptrón

checkpoint = ModelCheckpoint('best_model_perceptron.keras',
                             monitor='val_accuracy',
                             save_best_only=True,
                             mode='max',
                             verbose=1)

In [5]:
history = model_perceptron.fit(x_train, y_train,
                    epochs=30,
                    batch_size=64,
                    validation_data=(x_test, y_test),
                    callbacks=[checkpoint])

Epoch 1/30
380/438 ━━━━━━━━━━━━━━━━━━━━ 0s 801us/step - accuracy: 0.2413 - loss: 1.8574
Epoch 1: val_accuracy improved from -inf to 0.25029, saving model to best_model_perceptron.keras
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.2425 - loss: 1.8526 - val_accuracy: 0.2503 - val_loss: 1.8155
Epoch 2/30
433/438 ━━━━━━━━━━━━━━━━━━━━ 0s 817us/step - accuracy: 0.2505 - loss: 1.8105
Epoch 2: val_accuracy did not improve from 0.25029
438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 973us/step - accuracy: 0.2505 - loss: 1.8105 - val_accuracy: 0.2503 - val_loss: 1.8196
Epoch 3/30
417/438 ━━━━━━━━━━━━━━━━━━━━ 0s 867us/step - accuracy: 0.2519 - loss: 1.8065
Epoch 3: val_accuracy did not improve from 0.25029
438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2519 - loss: 1.8068 - val_accuracy: 0.2503 - val_loss: 1.8162
Epoch 4/30
393/438 ━━━━━━━━━━━━━━━━━━━━ 0s 904us/step - accuracy: 0.2526 - loss: 1.8074
Epoch 4: val_accuracy did not improve from 0.25029
438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - ac

# Para el ELM con pesos aleatorios no binarios

In [6]:
class elm(BaseEstimator, ClassifierMixin):
    def __init__(self, n_hidden_neurons=1000, regressor=None):
        self.n_hidden_neurons = n_hidden_neurons
        self.regressor = regressor

    def _sigmoid(self, x):
        return 1.0 / (1.0 + np.exp(-x))
    
    def fit(self, X, y, binary_weights = False):
        # Dependiendo de la selección de pesos binarios, se eligen pesos aleatorios o no
        input_size = X.shape[1]

        if binary_weights:
            self.input_weights = np.random.choice([-1, 1], size=(input_size, self.n_hidden_neurons))
            self.biases = np.random.choice([-1, 1], size=self.n_hidden_neurons)
        else:
            self.input_weights = np.random.randn(input_size, self.n_hidden_neurons)
            self.biases = np.random.randn(self.n_hidden_neurons)
        
        # Salida de la capa oculta
        H = self._sigmoid(np.dot(X, self.input_weights) + self.biases)
        
        # Regresión usando regularización si se especifica
        self.regressor.fit(H, y)

        # Calcular las predicciones sobre el conjunto de entrenamiento
        y_pred_train = self.predict(X)
        y_true_labels = np.argmax(y, axis=1)
        y_pred_labels = np.argmax(y_pred_train, axis=1)
        
        accuracy_train = accuracy_score(y_true_labels, y_pred_labels)
        print(f"Accuracy en el conjunto de entrenamiento: {accuracy_train:.4f}")
        return self
    
    def predict(self, X):
        H = self._sigmoid(np.dot(X, self.input_weights) + self.biases)
        return self.regressor.predict(H)
    
    def save(self, filename):
        # pa guardar el modelo
        model_data = {
            'input_weights': self.input_weights,
            'biases': self.biases,
            'regressor': self.regressor
        }
        with open(filename, 'wb') as f:
            pickle.dump(model_data, f)
    
    def load(self, filename):
        # pa cargar el modelo
        with open(filename, 'rb') as f:
            model_data = pickle.load(f)
        self.input_weights = model_data['input_weights']
        self.biases = model_data['biases']
        self.regressor = model_data['regressor']

## Para el caso del ELM con pesos no binarios y distintos tipos de regularización.

In [7]:
x_train_scaled = StandardScaler().fit_transform(x_train)

### Sin regularización.

In [8]:
# Sin regularización
elm_no_reg = elm(n_hidden_neurons=1000, regressor=Ridge(alpha=0.0))
elm_no_reg.fit(x_train_scaled, y_train)
elm_no_reg.save('elm_no_reg.keras')

Accuracy en el conjunto de entrenamiento: 0.3131


Es un accuracy aún más alto que con el perceptrón multicapa. 
### Con regularización Ridge

In [9]:
elm_ridge = elm(n_hidden_neurons=1000, regressor=Ridge(alpha=1.0)) 
elm_ridge.fit(x_train_scaled, y_train)
elm_ridge.save('elm_ridge.keras')

Accuracy en el conjunto de entrenamiento: 0.3150


Tiene una mejor cara que sin regularización.
### Con regularización Lasso

In [10]:
elm_lasso = elm(n_hidden_neurons=1000, regressor=Lasso(alpha=0.01))  
elm_lasso.fit(x_train_scaled, y_train)
elm_lasso.save('elm_lasso.keras')

Accuracy en el conjunto de entrenamiento: 0.2517


Con Lasso parece tener un desempeño similar al perceptrón multicapa.
### Con regularización Elastic-net.

In [11]:
elm_elastic = elm(n_hidden_neurons=1000, regressor=ElasticNet(alpha=0.01, l1_ratio=0.5))  
elm_elastic.fit(x_train_scaled, y_train)
elm_elastic.save('elm_elastic.keras')

Accuracy en el conjunto de entrenamiento: 0.2517


Lo mismo.

## Ahora con pesos binarios.

### Sin regularización

In [12]:
elm_no_reg_bin = elm(n_hidden_neurons=1000, regressor=Ridge(alpha=0.0))
elm_no_reg_bin.fit(x_train_scaled, y_train, binary_weights=True)
elm_no_reg_bin.save('elm_no_reg_bin.keras')

Accuracy en el conjunto de entrenamiento: 0.3157


Un poco mejor que su competencia con pesos continuos.
### Regularización Ridge.

In [13]:
elm_ridge_bin = elm(n_hidden_neurons=1000, regressor=Ridge(alpha=1.0))
elm_ridge_bin.fit(x_train_scaled, y_train, binary_weights=True)
elm_ridge_bin.save('elm_ridge_bin.keras')

Accuracy en el conjunto de entrenamiento: 0.3115


En este caso, la regularización parece no apoyar tanto.
### Regularización Lasso

In [14]:
elm_lasso_bin = elm(n_hidden_neurons=1000, regressor=Lasso(alpha=0.01))
elm_lasso_bin.fit(x_train_scaled, y_train, binary_weights=True)
elm_lasso_bin.save('elm_lasso_bin.keras')

Accuracy en el conjunto de entrenamiento: 0.2517


Apenas comparando con el perceptrón multicapa.
### Regularización Elastic-net

In [15]:
elm_elastic_bin = elm(n_hidden_neurons=1000, regressor=ElasticNet(alpha=0.01, l1_ratio=0.5))
elm_elastic_bin.fit(x_train_scaled, y_train, binary_weights=True)
elm_elastic_bin.save('elm_elastic_bin.keras')

Accuracy en el conjunto de entrenamiento: 0.2517


Parece ser que, al igual que en el caso de valores continuos, las últimas dos regularizaciones se comportan muy similar.